In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

C:\Users\iltya\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_table("train.tsv/train.tsv", delimiter="\t")

In [3]:
df.head()

,id,name,employer_name,experience_name,schedule_name,key_skills_name,accept_handicapped,accept_kids,unified_address_city,unified_address_state,...,raw_branded_description,lemmaized_wo_stopwords_raw_description,lemmaized_wo_stopwords_raw_branded_description,if_foreign_language,is_branded_description,name_clean,employment_name,employer_id,employer_industries,salary_mean_net
0,49242400,Инженер группы технической поддержки,DatsTeam,От 1 года до 3 лет,Сменный график,не указано,False,False,Москва,Москва,...,NaN,разрабатывать рекламный система оплата действи...,NaN,Не указано,заполнено,инженер группа технический поддержка,Полная занятость,4437201.0,не указано,80000.00
1,49496801,Специалист группы кадрового администрирования,Почта России,Нет опыта,Полный день,оформление трудовых книжек,False,False,Абакан,Республика Хакасия,...,АО «Почта России» входит в перечень системообр...,обязанность ведение кадровый делопроизводство ...,ао почта россия входить перечень системообразу...,Не указано,не заполнено,специалист группа кадровый администрирование,Полная занятость,4352.0,почтовая доставка,19989.12
2,44198403,Оператор 1С,Стройгарантсервис,От 1 года до 3 лет,Полный день,первичные документы,False,False,Москва,Москва,...,NaN,условие график работа 5/2 09.00 18.00;• оформл...,NaN,Не указано,заполнено,оператор 1с,Полная занятость,1794943.0,лифты,45000.00
3,47707600,Инженер ПТО,Коммунальник,От 3 до 6 лет,Полный день,autocad,False,False,NaN,Краснодарский край,...,NaN,строительный инженерный компания коммунальник ...,NaN,Не указано,заполнено,инженер пто,Полная занятость,1881241.0,не указано,40000.00
4,44610002,Водитель в войсковую часть (пункт отбора на во...,Пункт отбора на военную службу по контракту по...,Нет опыта,Полный день,водительское удостоверение категории bc,False,False,NaN,Калужская область,...,NaN,обязанность знать устройство правило эксплуата...,NaN,Не указано,заполнено,водитель войсковой часть пункт отбор военный с...,Полная занятость,1586217.0,не указано,37500.00


In [4]:
df.shape

(61314, 26)

## предобработка

In [5]:
df.nunique() / len(df)

id                                                1.000000
name                                              0.485370
employer_name                                     0.450908
experience_name                                   0.000065
schedule_name                                     0.000082
key_skills_name                                   0.047917
accept_handicapped                                0.000033
accept_kids                                       0.000033
unified_address_city                              0.003392
unified_address_state                             0.001370
unified_address_region                            0.000130
unified_address_country                           0.000016
specializations_profarea_name                     0.000473
professional_roles_name                           0.002381
languages_name                                    0.000832
raw_description                                   0.846919
raw_branded_description                           0.0685

In [6]:
df.drop("id", axis=1, inplace=True)
df.drop("employer_id", axis=1, inplace=True)
df.raw_branded_description.isna().sum() / len(df)

0.9189255308738624

In [7]:
df.drop("raw_branded_description", axis=1, inplace=True)
df["have_raw_branded_description"] = df["lemmaized_wo_stopwords_raw_branded_description"].isna()
df.drop("lemmaized_wo_stopwords_raw_branded_description", axis=1, inplace=True)

In [8]:
df["unified_address_city"] = df.unified_address_city.fillna("unkonown")
df["unified_address_region"] = df.unified_address_region.fillna("unknown")

In [9]:
df.head()

,name,employer_name,experience_name,schedule_name,key_skills_name,accept_handicapped,accept_kids,unified_address_city,unified_address_state,unified_address_region,...,languages_name,raw_description,lemmaized_wo_stopwords_raw_description,if_foreign_language,is_branded_description,name_clean,employment_name,employer_industries,salary_mean_net,have_raw_branded_description
0,Инженер группы технической поддержки,DatsTeam,От 1 года до 3 лет,Сменный график,не указано,False,False,Москва,Москва,Центральный федеральный округ,...,[],Мы разрабатываем рекламные системы с оплатой з...,разрабатывать рекламный система оплата действи...,Не указано,заполнено,инженер группа технический поддержка,Полная занятость,не указано,80000.00,True
1,Специалист группы кадрового администрирования,Почта России,Нет опыта,Полный день,оформление трудовых книжек,False,False,Абакан,Республика Хакасия,Сибирский федеральный округ,...,[],Обязанности: Ведение кадрового делопроизводств...,обязанность ведение кадровый делопроизводство ...,Не указано,не заполнено,специалист группа кадровый администрирование,Полная занятость,почтовая доставка,19989.12,False
2,Оператор 1С,Стройгарантсервис,От 1 года до 3 лет,Полный день,первичные документы,False,False,Москва,Москва,Центральный федеральный округ,...,[],Условия: • График работы 5/2 с 09.00 до 18.00;...,условие график работа 5/2 09.00 18.00;• оформл...,Не указано,заполнено,оператор 1с,Полная занятость,лифты,45000.00,True
3,Инженер ПТО,Коммунальник,От 3 до 6 лет,Полный день,autocad,False,False,unkonown,Краснодарский край,Южный федеральный округ,...,[],В строительную инженерную компанию «КОММУНАЛЬН...,строительный инженерный компания коммунальник ...,Не указано,заполнено,инженер пто,Полная занятость,не указано,40000.00,True
4,Водитель в войсковую часть (пункт отбора на во...,Пункт отбора на военную службу по контракту по...,Нет опыта,Полный день,водительское удостоверение категории bc,False,False,unkonown,Калужская область,Центральный федеральный округ,...,[],"Обязанности: - знать устройство, правила экспл...",обязанность знать устройство правило эксплуата...,Не указано,заполнено,водитель войсковой часть пункт отбор военный с...,Полная занятость,не указано,37500.00,True


In [10]:
df.isna().sum().sum()

0

In [11]:
df['is_branded_description'] = (df['is_branded_description'] == "заполнено")

In [12]:
df["if_foreign_language"] = (df["if_foreign_language"] == "Указано")

In [13]:
df["key_skills_name"] = df["key_skills_name"].apply(lambda x: x if x != "не указано" else "")

In [14]:
cat_features = ["experience_name", "schedule_name", "unified_address_city",
                "unified_address_region", "unified_address_state", "employment_name"]

In [15]:
for column in df.columns:
    print(column, len(set(df[column])))

name 29760
employer_name 27647
experience_name 4
schedule_name 5
key_skills_name 2938
accept_handicapped 2
accept_kids 2
unified_address_city 209
unified_address_state 84
unified_address_region 9
unified_address_country 1
specializations_profarea_name 29
professional_roles_name 146
languages_name 51
raw_description 51928
lemmaized_wo_stopwords_raw_description 51689
if_foreign_language 2
is_branded_description 2
name_clean 27998
employment_name 5
employer_industries 250
salary_mean_net 2929
have_raw_branded_description 2


In [16]:
import ast
all_lang = []

for langs in df["languages_name"].values:
    for i in set(ast.literal_eval(langs)):
        all_lang.append(i)

In [17]:
df["len_languages"] = [len(ast.literal_eval(i)) for i in df.languages_name]

In [18]:
def get_first_lang(string):
    lst = ast.literal_eval(string)
    if len(lst) > 0:
        return lst[0]
    return ""

def get_second_lang(string):
    lst = ast.literal_eval(string)
    if len(lst) > 1:
        return lst[1]
    return ""

In [19]:
tqdm.pandas()

In [20]:
df["first_lang"] = df["languages_name"].apply(get_first_lang)
df["second_lang"] = df["languages_name"].apply(get_second_lang)
cat_features.extend(["first_lang", "second_lang"])

In [21]:
df.drop("unified_address_country", axis=1, inplace=True)

In [22]:
df.drop("raw_description", axis=1, inplace=True)

In [23]:
df.drop("languages_name", axis=1, inplace=True)

In [24]:
df.drop("name", axis=1, inplace=True)

In [25]:
df.head()

,employer_name,experience_name,schedule_name,key_skills_name,accept_handicapped,accept_kids,unified_address_city,unified_address_state,unified_address_region,specializations_profarea_name,...,if_foreign_language,is_branded_description,name_clean,employment_name,employer_industries,salary_mean_net,have_raw_branded_description,len_languages,first_lang,second_lang
0,DatsTeam,От 1 года до 3 лет,Сменный график,,False,False,Москва,Москва,Центральный федеральный округ,информационные технологии,...,False,True,инженер группа технический поддержка,Полная занятость,не указано,80000.00,True,0,,
1,Почта России,Нет опыта,Полный день,оформление трудовых книжек,False,False,Абакан,Республика Хакасия,Сибирский федеральный округ,не указано,...,False,False,специалист группа кадровый администрирование,Полная занятость,почтовая доставка,19989.12,False,0,,
2,Стройгарантсервис,От 1 года до 3 лет,Полный день,первичные документы,False,False,Москва,Москва,Центральный федеральный округ,не указано,...,False,True,оператор 1с,Полная занятость,лифты,45000.00,True,0,,
3,Коммунальник,От 3 до 6 лет,Полный день,autocad,False,False,unkonown,Краснодарский край,Южный федеральный округ,строительство,...,False,True,инженер пто,Полная занятость,не указано,40000.00,True,0,,
4,Пункт отбора на военную службу по контракту по...,Нет опыта,Полный день,водительское удостоверение категории bc,False,False,unkonown,Калужская область,Центральный федеральный округ,не указано,...,False,True,водитель войсковой часть пункт отбор военный с...,Полная занятость,не указано,37500.00,True,0,,


In [26]:
df.key_skills_name

0                                               
1                     оформление трудовых книжек
2                            первичные документы
3                                        autocad
4        водительское удостоверение категории bc
                          ...                   
61309                                           
61310                                           
61311                         работа с клиентами
61312                                           
61313              навыки межличностного общения
Name: key_skills_name, Length: 61314, dtype: object

In [27]:
text_features = ["employer_name", "key_skills_name", "specializations_profarea_name",
                 "professional_roles_name", "lemmaized_wo_stopwords_raw_description",
                 "name_clean", "employer_industries"]

In [28]:
df.employer_industries

0                  не указано
1           почтовая доставка
2                       лифты
3                  не указано
4                  не указано
                 ...         
61309              не указано
61310              не указано
61311              не указано
61312              не указано
61313    общественное питание
Name: employer_industries, Length: 61314, dtype: object

## добавляем tfidf

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD


def vectorize_column(df, column_name, num_features=None, vectorizer=None, demension_reducer=None):
    df = df.copy()

    if vectorizer is None:
        vectorizer = TfidfVectorizer(max_features=num_features)
        res_emb = vectorizer.fit_transform(df[column_name]).toarray()
    else:
        res_emb = vectorizer.transform(df[column_name]).toarray()


    if demension_reducer is not None:
        res_emb = demension_reducer.fit_transform(res_emb)

    tfidf_inter_df = pd.DataFrame(res_emb, columns=[f"tfidf_{column_name}_{i}" for i in range(res_emb.shape[1])]).set_index(df.index)

    result_dataset = pd.concat([df, tfidf_inter_df], axis=1)

    return vectorizer, demension_reducer, result_dataset

In [30]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

dict_vectorizers = dict()
dict_pcas = dict()

for text_column in text_features:
    print(f"text column: {text_column}")
    vectorizer, pca, df = vectorize_column(df, text_column, num_features=100, demension_reducer=PCA(n_components=10))

    dict_vectorizers[text_column] = vectorizer
    dict_pcas[text_column] = pca

text column: employer_name
text column: key_skills_name
text column: specializations_profarea_name
text column: professional_roles_name
text column: lemmaized_wo_stopwords_raw_description
text column: name_clean
text column: employer_industries


## get bert embeddings

In [183]:
 df["full_text"] = df["employer_name"].fillna("") + " "  + df["key_skills_name"].fillna("") + " " + df["professional_roles_name"].fillna("")

In [184]:
import torch
from transformers import AutoTokenizer, AutoModel
device="cuda"
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny").to(device)
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [185]:
embeddings = np.array([embed_bert_cls(text, model, tokenizer) for text in tqdm(df.full_text)])

  0%|          | 0/61314 [00:00<?, ?it/s]

In [187]:
word2vec_pca = PCA(n_components=128)
new_emb = word2vec_pca.fit_transform(embeddings)

In [188]:
emb_df = pd.DataFrame(new_emb, columns=[f"word2vec_emb_pca_{i}" for i in range(new_emb.shape[1])])
df = pd.concat([df, emb_df], axis=1)

In [190]:
df.drop("full_text", axis=1, inplace=True)

## время тренировки

In [79]:
from sklearn.model_selection import  train_test_split
from catboost import Pool

X, y = df.drop("salary_mean_net", axis=1), df.salary_mean_net
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# y_train_log = np.log(y_train)
# y_test_log = np.log(y_test)

train_data = Pool(data=X_train, label=y_train, cat_features=cat_features, text_features=text_features)
test_data = Pool(data=X_test, label=y_test, cat_features=cat_features, text_features=text_features)


In [86]:
from catboost import CatBoostRegressor
clf = CatBoostRegressor(loss_function="RMSE",
                        eval_metric="MAPE", n_estimators=500,
                        task_type="GPU") # one_hot_max_size=209)

In [87]:
clf.fit(train_data, eval_set=test_data, early_stopping_rounds=100,)

Learning rate set to 0.116686
0:	learn: 0.5329429	test: 0.5278923	best: 0.5278923 (0)	total: 208ms	remaining: 1m 43s
1:	learn: 0.5102588	test: 0.5059965	best: 0.5059965 (1)	total: 315ms	remaining: 1m 18s
2:	learn: 0.4923522	test: 0.4887814	best: 0.4887814 (2)	total: 381ms	remaining: 1m 3s
3:	learn: 0.4751507	test: 0.4722409	best: 0.4722409 (3)	total: 449ms	remaining: 55.7s
4:	learn: 0.4608589	test: 0.4587869	best: 0.4587869 (4)	total: 522ms	remaining: 51.6s
5:	learn: 0.4482691	test: 0.4466489	best: 0.4466489 (5)	total: 593ms	remaining: 48.8s
6:	learn: 0.4391700	test: 0.4378883	best: 0.4378883 (6)	total: 645ms	remaining: 45.4s
7:	learn: 0.4275382	test: 0.4271155	best: 0.4271155 (7)	total: 715ms	remaining: 44s
8:	learn: 0.4189565	test: 0.4191945	best: 0.4191945 (8)	total: 777ms	remaining: 42.4s
9:	learn: 0.4111039	test: 0.4114568	best: 0.4114568 (9)	total: 826ms	remaining: 40.5s
10:	learn: 0.4056862	test: 0.4063685	best: 0.4063685 (10)	total: 865ms	remaining: 38.5s
11:	learn: 0.4000455	t

In [88]:
from sklearn.metrics import mean_absolute_percentage_error

In [89]:
mean_absolute_percentage_error(clf.predict(train_data), y_train)

0.22572628008539755

In [85]:
# mean_absolute_percentage_error(np.exp(clf.predict(test_data)), y_test)

## для теста

In [94]:
df_test = pd.read_table("test.tsv/test.tsv", delimiter="\t")

In [95]:
df_test.drop("id", axis=1, inplace=True)
df_test.drop("employer_id", axis=1, inplace=True)
df_test.drop("raw_branded_description", axis=1, inplace=True)
df_test["have_raw_branded_description"] = df_test["lemmaized_wo_stopwords_raw_branded_description"].isna()
df_test.drop("lemmaized_wo_stopwords_raw_branded_description", axis=1, inplace=True)

In [96]:
df_test["unified_address_city"] = df_test.unified_address_city.fillna("unkonown")
df_test["unified_address_region"] = df_test.unified_address_region.fillna("unknown")

In [97]:
df_test['is_branded_description'] = (df_test['is_branded_description'] == "заполнено")
df_test["if_foreign_language"] = (df_test["if_foreign_language"] == "Указано")
df_test["key_skills_name"] = df_test["key_skills_name"].apply(lambda x: x if x != "не указано" else "")

In [98]:
import ast
all_lang = []

for langs in df_test["languages_name"].values:
    for i in set(ast.literal_eval(langs)):
        all_lang.append(i)

In [99]:
df_test["len_languages"] = [len(ast.literal_eval(i)) for i in df_test.languages_name]

In [100]:
df_test["first_lang"] = df_test["languages_name"].apply(get_first_lang)
df_test["second_lang"] = df_test["languages_name"].apply(get_second_lang)

In [101]:
df_test.drop("unified_address_country", axis=1, inplace=True)
df_test.drop("raw_description", axis=1, inplace=True)
df_test.drop("languages_name", axis=1, inplace=True)
df_test.drop("name", axis=1, inplace=True)

In [103]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

for text_column in text_features:
    print(f"text column: {text_column}")
    _, _, df_test = vectorize_column(df_test, text_column,
                                           vectorizer=dict_vectorizers[text_column],
                                           demension_reducer=dict_pcas[text_column]
                                           )

text column: employer_name
text column: key_skills_name
text column: specializations_profarea_name
text column: professional_roles_name
text column: lemmaized_wo_stopwords_raw_description
text column: name_clean
text column: employer_industries


In [127]:
df_test

,employer_name,experience_name,schedule_name,key_skills_name,accept_handicapped,accept_kids,unified_address_city,unified_address_state,unified_address_region,specializations_profarea_name,...,tfidf_employer_industries_0,tfidf_employer_industries_1,tfidf_employer_industries_2,tfidf_employer_industries_3,tfidf_employer_industries_4,tfidf_employer_industries_5,tfidf_employer_industries_6,tfidf_employer_industries_7,tfidf_employer_industries_8,tfidf_employer_industries_9
0,билайн,Нет опыта,Полный день,активные продажи,False,False,unkonown,Красноярский край,Сибирский федеральный округ,не указано,...,-0.381053,-0.127319,-0.008873,0.001596,0.03747,-0.075132,-0.057673,-0.022209,0.014285,-0.041983
1,GSR РАБОТА,Нет опыта,Полный день,контроль отгрузки и доставки товара,False,False,Нижний Новгород,Нижегородская область,Приволжский федеральный округ,не указано,...,0.609203,0.021611,0.000456,0.000137,-0.00196,0.003128,0.001967,0.000484,0.000443,0.000619
2,Супермен (ИП Титов Кирилл Анатольевич),Нет опыта,Полный день,,False,False,Самара,Самарская область,Приволжский федеральный округ,не указано,...,0.609203,0.021611,0.000456,0.000137,-0.00196,0.003128,0.001967,0.000484,0.000443,0.000619
3,Потапов Леонид Викторович,От 1 года до 3 лет,Полный день,грамотность,False,False,Москва,Москва,Центральный федеральный округ,не указано,...,-0.380115,-0.126046,-0.008947,-0.000882,0.03815,-0.072557,-0.062486,-0.015449,0.007980,-0.042207
4,Работут,Нет опыта,Сменный график,сканер,False,False,Химки,Московская область,Центральный федеральный округ,рабочий персонал,...,0.609203,0.021611,0.000456,0.000137,-0.00196,0.003128,0.001967,0.000484,0.000443,0.000619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26256,А101-Комфорт,От 3 до 6 лет,Полный день,руководство коллективом,False,False,Москва,Москва,Центральный федеральный округ,строительство,...,-0.369298,-0.112179,-0.006077,-0.001834,0.02849,-0.051126,-0.038961,-0.011879,-0.011623,-0.018012
26257,Областное бюджетное учреждение здравоохранения...,Нет опыта,Полный день,консультирование,False,False,Иваново,Ивановская область,Центральный федеральный округ,не указано,...,0.609203,0.021611,0.000456,0.000137,-0.00196,0.003128,0.001967,0.000484,0.000443,0.000619
26258,Единый Центр,От 1 года до 3 лет,Полный день,арбитражные суды,False,False,unkonown,Красноярский край,Сибирский федеральный округ,не указано,...,0.609203,0.021611,0.000456,0.000137,-0.00196,0.003128,0.001967,0.000484,0.000443,0.000619
26259,СИТИЛИНК,Нет опыта,Сменный график,проведение инвентаризаций,False,False,Москва,Москва,Центральный федеральный округ,не указано,...,-0.394951,-0.149393,-0.012891,-0.005045,0.07301,-0.166941,-0.205687,0.794533,0.215852,0.177554


In [105]:
test_pool = Pool(data=df_test, cat_features=cat_features, text_features=text_features)

In [107]:
answers = clf.predict(test_pool)

In [123]:
sample_sub = pd.read_csv("sample_submission.tsv", delimiter="\t")

In [124]:
sample_sub.head()

,id,salary_mean_net
0,45165602,77508
1,47291202,409890
2,44208002,162591
3,43765603,179264
4,49313000,482511


In [115]:
pd.read_csv("test.tsv/test.tsv", delimiter="\t").id == sample_sub.id

0        True
1        True
2        True
3        True
4        True
         ... 
26256    True
26257    True
26258    True
26259    True
26260    True
Name: id, Length: 26261, dtype: bool

In [116]:
sample_sub["salary_mean_net"] = answers

In [125]:
sample_sub.to_csv("baseline_v1.csv", index=None, sep="\t")

- добавить w2vec
- посмотреть отличия с решением оргов - почему у них простейшее дерево рандомное выбивает больше
- повыбивать макс скор
- залить что есть